# data preprocessing

In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb

from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
''' 
train_df = train_label + train_data
test_df = test_label + test_data

all_df = all_label + all_data
'''

dir_path = "/data/ephemeral/home/data"

train_label = pd.read_csv(f"{dir_path}/train.csv").assign(_type="train")
test_label = pd.read_csv(f"{dir_path}/test.csv").assign(_type="test")
print(train_label.shape)
print(test_label.shape)

file_names = glob(f"{dir_path}/HOURLY_*.csv")
print(len(file_names))    # data 개수

all_label = pd.concat([train_label, test_label], axis=0)

all_df = pd.concat([train_label, test_label], axis=0)

for i in tqdm(range(len(file_names))):
    dff = pd.read_csv(file_names[i])
    # columns 변경
    fname = file_names[i].split('/')[-1].split('.')[0].lower()   # 소문자 file name
    new_col = dff.columns.tolist()[1:]
    dff.columns = ['ID'] + [f"{fname}_{col}" for col in new_col]

    all_df = all_df.merge(dff, on='ID', how='left')

all_df.reset_index(drop=True, inplace=True)
print(all_df.shape)
all_df

(8760, 3)
(2792, 2)
107


100%|██████████| 107/107 [00:07<00:00, 13.89it/s]

(11552, 255)


,ID,target,_type,hourly_network-data_block-count_block_count,hourly_market-data_open-interest_bitmex_all_symbol_open_interest,hourly_market-data_liquidations_binance_btc_usd_long_liquidations,hourly_market-data_liquidations_binance_btc_usd_short_liquidations,hourly_market-data_liquidations_binance_btc_usd_long_liquidations_usd,hourly_market-data_liquidations_binance_btc_usd_short_liquidations_usd,hourly_market-data_liquidations_gate_io_btc_usd_long_liquidations,...,hourly_market-data_funding-rates_bitmex_funding_rates,hourly_market-data_taker-buy-sell-stats_deribit_taker_buy_volume,hourly_market-data_taker-buy-sell-stats_deribit_taker_sell_volume,hourly_market-data_taker-buy-sell-stats_deribit_taker_buy_ratio,hourly_market-data_taker-buy-sell-stats_deribit_taker_sell_ratio,hourly_market-data_taker-buy-sell-stats_deribit_taker_buy_sell_ratio,hourly_market-data_liquidations_ftx_all_symbol_long_liquidations,hourly_market-data_liquidations_ftx_all_symbol_short_liquidations,hourly_market-data_liquidations_ftx_all_symbol_long_liquidations_usd,hourly_market-data_liquidations_ftx_all_symbol_short_liquidations_usd
0,2023-01-01 00:00:00,2.0,train,12.0,166608500.0,0.0,0.0,0.0,0.0,0.0,...,0.001400,377290.0,252550.0,0.599025,0.400975,1.493922,NaN,NaN,NaN,NaN
1,2023-01-01 01:00:00,1.0,train,4.0,166630200.0,0.0,0.0,0.0,0.0,0.0,...,0.001400,463720.0,116960.0,0.798581,0.201419,3.964774,NaN,NaN,NaN,NaN
2,2023-01-01 02:00:00,1.0,train,8.0,167253600.0,0.0,0.0,0.0,0.0,0.0,...,0.001400,172790.0,603920.0,0.222464,0.777536,0.286114,NaN,NaN,NaN,NaN
3,2023-01-01 03:00:00,1.0,train,5.0,167626100.0,0.0,0.0,0.0,0.0,0.0,...,0.001518,176450.0,199630.0,0.469182,0.530818,0.883885,NaN,NaN,NaN,NaN
4,2023-01-01 04:00:00,2.0,train,7.0,168749700.0,0.0,0.0,0.0,0.0,0.0,...,0.008400,130040.0,377920.0,0.256004,0.743996,0.344094,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,test,3.0,296152100.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11548,2024-04-26 04:00:00,NaN,test,NaN,295864300.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11549,2024-04-26 05:00:00,NaN,test,NaN,294699200.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11550,2024-04-26 06:00:00,NaN,test,NaN,294335300.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# 결측치 100% 칼럼 제외
train_df = all_df.loc[all_df["_type"] == "train"]
missing_values = train_df.isnull().sum()
missing_percentage = (missing_values / len(train_df)) * 100
missing_lst = missing_percentage[missing_percentage == 100].index.tolist()
print(len(missing_lst))

all_df = all_df.drop(columns=missing_lst)
print(all_df.shape)

40
(11552, 215)


In [4]:
# Base 코드의 칼럼 선택
cols_dict = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_gap": "coinbase_premium_gap",
    "hourly_market-data_coinbase-premium-index_coinbase_premium_index": "coinbase_premium_index",
    "hourly_market-data_funding-rates_all_exchange_funding_rates": "funding_rates",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations": "long_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd": "long_liquidations_usd",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations": "short_liquidations",
    "hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd": "short_liquidations_usd",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_ratio": "buy_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_sell_ratio": "buy_sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_buy_volume": "buy_volume",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_ratio": "sell_ratio",
    "hourly_market-data_taker-buy-sell-stats_all_exchange_taker_sell_volume": "sell_volume",
    "hourly_network-data_addresses-count_addresses_count_active": "active_count",
    "hourly_network-data_addresses-count_addresses_count_receiver": "receiver_count",
    "hourly_network-data_addresses-count_addresses_count_sender": "sender_count",
}
print(all_df.shape)

all_df = all_df[cols_dict.keys()].rename(cols_dict, axis=1)
print(all_df.shape)

all_df = all_df.assign(
    liquidation_diff=all_df["long_liquidations"] - all_df["short_liquidations"],
    liquidation_usd_diff=all_df["long_liquidations_usd"] - all_df["short_liquidations_usd"],
    volume_diff=all_df["buy_volume"] - all_df["sell_volume"],
    liquidation_diffg=np.sign(all_df["long_liquidations"] - all_df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(all_df["long_liquidations_usd"] - all_df["short_liquidations_usd"]),
    volume_diffg=np.sign(all_df["buy_volume"] - all_df["sell_volume"]),
    buy_sell_volume_ratio=all_df["buy_volume"] / (all_df["sell_volume"] + 1),
)
print(all_df.shape)

(11552, 215)
(11552, 19)
(11552, 26)


In [5]:
all_df

,ID,target,_type,coinbase_premium_gap,coinbase_premium_index,funding_rates,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,...,active_count,receiver_count,sender_count,liquidation_diff,liquidation_usd_diff,volume_diff,liquidation_diffg,liquidation_usd_diffg,volume_diffg,buy_sell_volume_ratio
0,2023-01-01 00:00:00,2.0,train,-9.86,-0.059650,0.005049,0.012000,197.51610,0.000000,0.00000,...,67987,37752,37307,0.012000,197.51610,-4.892284e+06,1.0,1.0,-1.0,0.904774
1,2023-01-01 01:00:00,1.0,train,-8.78,-0.053047,0.005049,0.000000,0.00000,0.712000,11833.56104,...,30593,20534,12342,-0.712000,-11833.56104,1.574470e+07,-1.0,-1.0,1.0,1.655721
2,2023-01-01 02:00:00,1.0,train,-9.59,-0.057952,0.005049,0.000000,0.00000,0.000000,0.00000,...,33897,19369,17737,0.000000,0.00000,6.440658e+05,0.0,0.0,1.0,1.027512
3,2023-01-01 03:00:00,1.0,train,-9.74,-0.058912,0.005067,0.593000,9754.76891,0.000000,0.00000,...,32717,23799,11421,0.593000,9754.76891,-4.105574e+06,1.0,1.0,-1.0,0.874477
4,2023-01-01 04:00:00,2.0,train,-10.14,-0.061373,0.006210,0.361000,5944.43714,0.000000,0.00000,...,45176,31712,17320,0.361000,5944.43714,-1.060461e+06,1.0,1.0,-1.0,0.966796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,test,-1.53,-0.002376,NaN,0.710000,45484.20433,0.243500,15682.76464,...,29250,13601,18154,0.466500,29801.43969,NaN,1.0,1.0,NaN,NaN
11548,2024-04-26 04:00:00,NaN,test,-11.73,-0.018268,NaN,6.577208,420718.03779,0.146000,9419.65430,...,56580,29096,31320,6.431208,411298.38349,NaN,1.0,1.0,NaN,NaN
11549,2024-04-26 05:00:00,NaN,test,1.85,0.002866,NaN,1.797163,114902.59095,5.216490,337367.12807,...,51858,22094,34083,-3.419327,-222464.53712,NaN,-1.0,-1.0,NaN,NaN
11550,2024-04-26 06:00:00,NaN,test,-2.05,-0.003184,NaN,0.803000,51434.51531,1.656000,106931.54104,...,36270,12668,26186,-0.853000,-55497.02573,NaN,-1.0,-1.0,NaN,NaN


In [6]:
# columns list for feature
col_lst = all_df.drop(columns=['ID', 'target', '_type']).columns.tolist()
col_lst

['coinbase_premium_gap',
 'coinbase_premium_index',
 'funding_rates',
 'long_liquidations',
 'long_liquidations_usd',
 'short_liquidations',
 'short_liquidations_usd',
 'open_interest',
 'buy_ratio',
 'buy_sell_ratio',
 'buy_volume',
 'sell_ratio',
 'sell_volume',
 'active_count',
 'receiver_count',
 'sender_count',
 'liquidation_diff',
 'liquidation_usd_diff',
 'volume_diff',
 'liquidation_diffg',
 'liquidation_usd_diffg',
 'volume_diffg',
 'buy_sell_volume_ratio']

In [7]:
all_df_aug = all_df.copy()
print("all_df_aug shape: ", all_df_aug.shape)

# rolling 적용
w_lst = [i for i in range(1, 11)]
for col in tqdm(col_lst):
    for w in w_lst:
        new_col = f"{col}_rolling_mean_{w}"
        all_df_aug[new_col] = all_df_aug[col].rolling(window=w).mean()
        new_col = f"{col}_rolling_std_{w}"
        all_df_aug[new_col] = all_df_aug[col].rolling(window=w).std()
print("all_df_aug shape: ", all_df_aug.shape)

# shift 적용
p_lst = [i for i in range(1, 25)]
for col in tqdm(col_lst):
    for p in p_lst:
        new_col = f"{col}_shift_{w}"
        all_df_aug[new_col] = all_df_aug[col].shift(periods=p)
print("all_df_aug shape: ", all_df_aug.shape)

# 타겟 변수를 제외한 변수를 forwardfill, -999로 결측치 대체
_target = all_df_aug["target"]
all_df_aug = all_df_aug.ffill().fillna(-999).assign(target = _target)

all_df_aug shape:  (11552, 26)


100%|██████████| 23/23 [00:00<00:00, 51.67it/s]


all_df_aug shape:  (11552, 486)


100%|██████████| 23/23 [00:00<00:00, 207.44it/s]


all_df_aug shape:  (11552, 509)


In [8]:
# Save
all_df_aug.to_csv("./data.csv", index=False)
print(all_df_aug.shape)

train_df = all_df_aug.loc[all_df_aug["_type"]=="train"].drop(columns=["_type"])
test_df = all_df_aug.loc[all_df_aug["_type"]=="test"].drop(columns=["_type"])
print("train shape: ", train_df.shape)
print("test shape: ", test_df.shape)

(11552, 509)
train shape:  (8760, 508)
test shape:  (2792, 508)


In [9]:
train_df

,ID,target,coinbase_premium_gap,coinbase_premium_index,funding_rates,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,open_interest,...,active_count_shift_10,receiver_count_shift_10,sender_count_shift_10,liquidation_diff_shift_10,liquidation_usd_diff_shift_10,volume_diff_shift_10,liquidation_diffg_shift_10,liquidation_usd_diffg_shift_10,volume_diffg_shift_10,buy_sell_volume_ratio_shift_10
0,2023-01-01 00:00:00,2.0,-9.86,-0.059650,0.005049,0.012000,1.975161e+02,0.0000,0.000000e+00,6.271344e+09,...,-999.0,-999.0,-999.0,-999.000000,-9.990000e+02,-9.990000e+02,-999.0,-999.0,-999.0,-999.000000
1,2023-01-01 01:00:00,1.0,-8.78,-0.053047,0.005049,0.000000,0.000000e+00,0.7120,1.183356e+04,6.288683e+09,...,-999.0,-999.0,-999.0,-999.000000,-9.990000e+02,-9.990000e+02,-999.0,-999.0,-999.0,-999.000000
2,2023-01-01 02:00:00,1.0,-9.59,-0.057952,0.005049,0.000000,0.000000e+00,0.0000,0.000000e+00,6.286796e+09,...,-999.0,-999.0,-999.0,-999.000000,-9.990000e+02,-9.990000e+02,-999.0,-999.0,-999.0,-999.000000
3,2023-01-01 03:00:00,1.0,-9.74,-0.058912,0.005067,0.593000,9.754769e+03,0.0000,0.000000e+00,6.284575e+09,...,-999.0,-999.0,-999.0,-999.000000,-9.990000e+02,-9.990000e+02,-999.0,-999.0,-999.0,-999.000000
4,2023-01-01 04:00:00,2.0,-10.14,-0.061373,0.006210,0.361000,5.944437e+03,0.0000,0.000000e+00,6.291582e+09,...,-999.0,-999.0,-999.0,-999.000000,-9.990000e+02,-9.990000e+02,-999.0,-999.0,-999.0,-999.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,1.0,-27.10,-0.063587,0.036371,0.163000,6.924677e+03,0.1040,4.437410e+03,1.054230e+10,...,66588.0,25216.0,45173.0,-0.244000,-1.042350e+04,-6.817628e+06,-1.0,-1.0,-1.0,0.926607
8756,2023-12-31 20:00:00,1.0,-24.73,-0.058109,0.037233,29.698896,1.263031e+06,43.8396,1.870481e+06,1.051484e+10,...,56094.0,18138.0,40443.0,25.660994,1.080277e+06,-5.045491e+07,1.0,1.0,-1.0,0.727422
8757,2023-12-31 21:00:00,0.0,-28.48,-0.066979,0.037761,0.325000,1.385218e+04,1.6790,7.179552e+04,1.048598e+10,...,101178.0,35250.0,70576.0,0.366000,1.540562e+04,3.671082e+06,1.0,1.0,1.0,1.038351
8758,2023-12-31 22:00:00,2.0,-9.08,-0.021487,0.038020,90.293123,3.815777e+06,7.7816,3.310213e+05,1.032844e+10,...,67738.0,22928.0,48233.0,34.359051,1.443065e+06,-5.371714e+07,1.0,1.0,-1.0,0.610975


In [10]:
test_df

,ID,target,coinbase_premium_gap,coinbase_premium_index,funding_rates,long_liquidations,long_liquidations_usd,short_liquidations,short_liquidations_usd,open_interest,...,active_count_shift_10,receiver_count_shift_10,sender_count_shift_10,liquidation_diff_shift_10,liquidation_usd_diff_shift_10,volume_diff_shift_10,liquidation_diffg_shift_10,liquidation_usd_diffg_shift_10,volume_diffg_shift_10,buy_sell_volume_ratio_shift_10
8760,2024-01-01 00:00:00,NaN,-22.57,-0.053137,0.042276,0.661000,27996.83627,17.251477,7.350195e+05,1.034554e+10,...,80380.0,25484.0,58539.0,-3.275801,-1.393753e+05,6.311677e+07,-1.0,-1.0,1.0,1.475138
8761,2024-01-01 01:00:00,NaN,-18.88,-0.044305,0.042578,0.483000,20552.07715,36.152847,1.546929e+06,1.040165e+10,...,66899.0,17439.0,51709.0,1.924824,8.087924e+04,-4.717319e+07,1.0,1.0,-1.0,0.701483
8762,2024-01-01 02:00:00,NaN,-9.78,-0.022968,0.043265,6.724000,285861.94051,1.623000,6.941107e+04,1.039594e+10,...,65812.0,15247.0,52548.0,35.532924,1.493391e+06,-6.240264e+07,1.0,1.0,-1.0,0.740465
8763,2024-01-01 03:00:00,NaN,-5.38,-0.012710,0.043927,20.107080,849377.45559,0.071000,3.003284e+03,1.033749e+10,...,41020.0,15500.0,26760.0,-0.651000,-2.787925e+04,3.778580e+07,-1.0,-1.0,1.0,1.495168
8764,2024-01-01 04:00:00,NaN,-10.22,-0.024104,0.045189,12.654933,533700.00186,1.651000,7.014376e+04,1.039205e+10,...,57263.0,20099.0,38653.0,-5.000800,-2.122320e+05,2.343655e+07,-1.0,-1.0,1.0,1.234126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,NaN,-1.53,-0.002376,0.009411,0.710000,45484.20433,0.243500,1.568276e+04,1.486836e+10,...,30621.0,12120.0,21420.0,-0.093883,-6.829373e+03,6.484229e+07,-1.0,-1.0,1.0,1.253965
11548,2024-04-26 04:00:00,NaN,-11.73,-0.018268,0.009411,6.577208,420718.03779,0.146000,9.419654e+03,1.486836e+10,...,44129.0,25097.0,22240.0,0.448671,2.813475e+04,6.484229e+07,1.0,1.0,1.0,1.253965
11549,2024-04-26 05:00:00,NaN,1.85,0.002866,0.009411,1.797163,114902.59095,5.216490,3.373671e+05,1.486836e+10,...,67914.0,24590.0,48084.0,0.166541,9.996741e+03,6.484229e+07,1.0,1.0,1.0,1.253965
11550,2024-04-26 06:00:00,NaN,-2.05,-0.003184,0.009411,0.803000,51434.51531,1.656000,1.069315e+05,1.486836e+10,...,50572.0,16599.0,37638.0,-0.087000,-5.727296e+03,6.484229e+07,-1.0,-1.0,1.0,1.253965
